In [1]:
!pip freeze | grep tensorflow==2.5

tensorflow==2.5.0


In [3]:
# Import libraries
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

2.5.0


In [4]:
# Read the data

CSV_COLUMNS = ['fare_amount','pickuplon','pickuplat','dropofflon','dropofflat','passengers','key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, mode, batch_size=512):
    def _input_fn():
        def decode_csv(value_column):
            columns = tf.compat.v1.decode_csv(value_column, record_defaults=DEFAULTS)
            features = dict(zip(CSV_COLUMNS, columns))
            label = features.pop(LABEL_COLUMN)
            return features, label
        
        # Create list of files that match pattern
        file_list = tf.compat.v1.gfile.Glob(filename)
        
        # Create dataset from file list
        dataset = tf.compat.v1.data.TextLineDataset(file_list).map(decode_csv)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinetely
            dataset = dataset.shuffle(buffer_size=10*batch_size)
        else:
            num_epochs=1 # end-of-input after this
            
        dataset = dataset.repeat(num_epochs).batch(batch_size)
        return dataset.make_one_shot_iterator().get_next()
    return _input_fn

In [5]:
# Create features out of input data
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickuplon'),
    tf.feature_column.numeric_column('pickuplat'),
    tf.feature_column.numeric_column('dropofflat'),
    tf.feature_column.numeric_column('dropofflon'),
    tf.feature_column.numeric_column('passengers')
]
def add_more_features(feats):
    # Nothing to add (yet!)
    return feats

features_cols = add_more_features(INPUT_COLUMNS)

In [6]:
# Train and evaluate
def service_input_fn():
    feature_placeholders = {
        'pickuplon' : tf.compat.v1.placeholder(tf.float32, [None]),
        'pickuplat' : tf.compat.v1.placeholder(tf.float32, [None]),
        'dropofflat' : tf.compat.v1.placeholder(tf.float32, [None]),
        'dropofflon' : tf.compat.v1.placeholder(tf.float32, [None]),
        'passengers' : tf.compat.v1.placeholder(tf.float32, [None])
    }
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [7]:
def train_and_evaluate(output_dir, num_train_steps):
    estimator = tf.estimator.LinearRegressor(
        model_dir=output_dir,
        feature_columns=features_cols)
    train_spec=tf.estimator.TrainSpec(
        input_fn=read_dataset('./taxi-train.csv', 
                              mode=tf.estimator.ModeKeys.TRAIN),
        max_steps=num_train_steps)
    exporter = tf.estimator.LatestExporter('exporter', service_input_fn)
    eval_spec=tf.estimator.EvalSpec(input_fn=read_dataset('./taxi-valid.csv',
                                                          mode=tf.estimator.ModeKeys.EVAL), 
                                    steps=None, 
                                    start_delay_secs=1, # start evaluating after N seconds
                                    throttle_secs=10, # evaluate every N seconds
                                    exporters=exporter)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [8]:
# Run training
OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors=True) # start fresh each time
train_and_evaluate(OUTDIR, num_train_steps=5000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'taxi_trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training an

2022-08-21 13:46:01.306783: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-21 13:46:01.306816: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-21 13:46:01.306902: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vadzim-pc): /proc/driver/nvidia/version does not exist
/home/vadzim/miniconda3/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


2022-08-21 13:46:02.034470: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-21 13:46:02.134176: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2596310000 Hz


INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into taxi_trained/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 201.44138, step = 0


2022-08-21 13:46:03.131698: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


INFO:tensorflow:global_step/sec: 74.7138
INFO:tensorflow:loss = 69.58467, step = 100 (1.340 sec)
INFO:tensorflow:global_step/sec: 89.7481
INFO:tensorflow:loss = 72.18041, step = 200 (1.114 sec)
INFO:tensorflow:global_step/sec: 83.0735
INFO:tensorflow:loss = 99.286865, step = 300 (1.205 sec)
INFO:tensorflow:global_step/sec: 89.7447
INFO:tensorflow:loss = 100.000565, step = 400 (1.113 sec)
INFO:tensorflow:global_step/sec: 88.1101
INFO:tensorflow:loss = 92.08533, step = 500 (1.135 sec)
INFO:tensorflow:global_step/sec: 81.0963
INFO:tensorflow:loss = 110.32201, step = 600 (1.233 sec)
INFO:tensorflow:global_step/sec: 90.3867
INFO:tensorflow:loss = 81.94925, step = 700 (1.107 sec)
INFO:tensorflow:global_step/sec: 89.0936
INFO:tensorflow:loss = 93.9918, step = 800 (1.122 sec)
INFO:tensorflow:global_step/sec: 89.2221
INFO:tensorflow:loss = 103.135666, step = 900 (1.121 sec)
INFO:tensorflow:global_step/sec: 88.9828
INFO:tensorflow:loss = 75.616684, step = 1000 (1.125 sec)
INFO:tensorflow:global_